# Kustuo 101 - Python SDK and Kqlmagic

#### The Python SDK allows you to query Kusto clusters and ingest data from Kusto for pipelining purposes. For this tutorial, we are only concerned with the azure-kusto-data module. 
Documentation [here](https://github.com/Azure/azure-kusto-python)

## Python SDK

#### Install Azure Log Analytics Package

In [ ]:
#install python kusto module, allows us to connect to kusto databases
# use this syntax to update a current version 
!pip install --upgrade azure-kusto-data

#use this syntax to install for the first time 
!pip install azure-kusto-data

In [ ]:
!pip install azure-common 
! pip install azure-mgmt-sql
! pip install azure-mgmt-resource
! pip install azure-mgmt-rdbms
! pip install azure-storage

In [1]:
import pandas as pd
import numpy as np
from azure.kusto.data.request import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

#### Set the cluster, create a connection string, create a client connection, and specify the database to work in

In [2]:
cluster = "https://help.kusto.windows.net"
kcsb = KustoConnectionStringBuilder.with_aad_device_authentication(cluster)
client = KustoClient(kcsb)
db = "Samples" 
#specify the database, in this case Samples containes the StormEvents table. Setting as a variable but you could do this explicitly in the execute function

##### Running the client.execute() code below will open a window which prompts you for a code which should appear below. Once entered you should be able to use kusto in Python.
##### Note!: you will have to do this every time you run Kusto in your notebook, but only once per Kernel. This is only for this particular cluster. 

In [3]:
query = "StormEvents | take 10" #write out the Kusto query with no line breaks

response = client.execute(db, query) #this executes the query

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BNE8P7SKP to authenticate.


#### The "response" variable is a kusto type object and isn't in a classic pandas dataframe

In [4]:
#you can see that "response" is a kusto response data table
response

In [5]:
#a new feature added by the Kusto Python SDK allows you to convert a kusto data table from a query into a pandas dataframe
df = dataframe_from_result_table(response.primary_results[0])

In [6]:
df.head(3)

,StartTime,EndTime,EpisodeId,EventId,State,EventType,InjuriesDirect,InjuriesIndirect,DeathsDirect,DeathsIndirect,...,Source,BeginLocation,EndLocation,BeginLat,BeginLon,EndLat,EndLon,EpisodeNarrative,EventNarrative,StormSummary
0,2007-09-29 08:11:00,2007-09-29 08:11:00,11091,61032,ATLANTIC SOUTH,Waterspout,0,0,0,0,...,Trained Spotter,MELBOURNE BEACH,MELBOURNE BEACH,28.0393,-80.6048,28.0393,-80.6048,Showers and thunderstorms lingering along the ...,A waterspout formed in the Atlantic southeast ...,"{'TotalDamages': 0, 'StartTime': '2007-09-29T0..."
1,2007-09-18 20:00:00,2007-09-19 18:00:00,11074,60904,FLORIDA,Heavy Rain,0,0,0,0,...,Trained Spotter,ORMOND BEACH,NEW SMYRNA BEACH,29.2800,-81.0500,29.0200,-80.9300,Thunderstorms lingered over Volusia County.,As much as 9 inches of rain fell in a 24-hour ...,"{'TotalDamages': 0, 'StartTime': '2007-09-18T2..."
2,2007-09-20 21:57:00,2007-09-20 22:05:00,11078,60913,FLORIDA,Tornado,0,0,0,0,...,NWS Storm Survey,EUSTIS,EUSTIS,28.8368,-81.6763,28.8628,-81.6846,As Tropical Depression number 10 began to slow...,A tornado touched down in the Town of Eustis a...,"{'TotalDamages': 6200000, 'StartTime': '2007-0..."


#### Quick Tip: working with Dynamic fields in Kusto Python SDK 

#### Note we are taking a slightly different approach to connecting to the Kusto Cluster

In [7]:
#change cluster 
cga = "https://cgadataout.kusto.windows.net:443"
cga_conn = KustoConnectionStringBuilder.with_aad_device_authentication(cga)
cga_client = KustoClient(cga_conn)
#creating a client that specifically connects to the CGA cluster, makes it easier if working with different clusters
cga_client.execute("CustomerModel", ".show version") 
#de-couples the execute and query, this way the SDK doesn't have to confirm the connection to the cluster every time a query is executed
cga_db = 'CustomerModel'

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BZV9FYU29 to authenticate.


In [8]:
#this query contains a Dynamic Kusto data field
kusto_json = "vwAllCustomerActivity(ago(900d), now()) | where ActivityClassId == 'PluralSightActivities'" 

#executes a KQL statement and stores it as a KQL data table
#note that we use "execute_query" rather than execute, since we are already connected to the cluster 
table_with_json = cga_client.execute_query(cga_db, kusto_json) 

In [9]:
#this code converts the dynamic field in the Kusto query into a dataframe
# the dynamic field still remains a JSON like data type  
df2 = dataframe_from_result_table(table_with_json.primary_results[0])

In [10]:
df2.head(3)

,Timeline,CustomerKeyType,CustomerKeyValue,ActivityClassId,Properties
0,2017-09-28,ANID,0B538A0E96F391FF3FDF0AC1FFFFFFFF,PluralSightActivities,{'CourseName': 'Managing Infrastructure with M...
1,2017-09-28,ANID,0B538A0E96F391FF3FDF0AC1FFFFFFFF,PluralSightActivities,{'CourseName': 'Managing Infrastructure with M...
2,2017-09-28,ANID,0B538A0E96F391FF3FDF0AC1FFFFFFFF,PluralSightActivities,{'CourseName': 'Managing Infrastructure with M...


In [11]:
#Once it's in a dataframe we can parse the JSON object like so
dfjson = pd.concat([df2.drop(['Properties'], axis=1), df2['Properties'].apply(pd.Series)], axis=1)  
#Note: prior versions of the SDK didn't handle KQL that parsed dynamic Kusto data types...
#hopefully they add that functionality soon because this method can take several seconds if parsing 100K+ rows
#If someone wants to see if this version does let me know and I'll buy you coffee

In [13]:
dfjson.head(3)

,Timeline,CustomerKeyType,CustomerKeyValue,ActivityClassId,CourseName,ModuleName,OfferCode,ViewTimeSeconds
0,2017-09-28,ANID,0B538A0E96F391FF3FDF0AC1FFFFFFFF,PluralSightActivities,Managing Infrastructure with Microsoft Azure -...,Course Overview,668-1-UP7V-ABRD,100
1,2017-09-28,ANID,0B538A0E96F391FF3FDF0AC1FFFFFFFF,PluralSightActivities,Managing Infrastructure with Microsoft Azure -...,Disks and Storage,668-1-UP7V-ABRD,283
2,2017-09-28,ANID,0B538A0E96F391FF3FDF0AC1FFFFFFFF,PluralSightActivities,Managing Infrastructure with Microsoft Azure -...,Foundational Concepts,668-1-UP7V-ABRD,1612


## Kqlmagic: Jupyter Notebook or Jupyter Lab Plugin for Kusto

### [Link to Kqlmagic Starter Notebook](https://mybinder.org/v2/gh/Microsoft/jupyter-Kqlmagic/master?filepath=notebooks%2FQuickStart.ipynb)